In [ ]:
import time
import sys
sys.path.append('../utils')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import stellargraph as sg

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [ ]:
# Load data

from load_data import load_data
v_data, e_data, core_target, ext_target, core_testing = load_data()

In [ ]:
# Subsample dataset

from tools import subsample
v_sample, e_sample, core_sample, ext_sample = subsample(v_data, e_data, core_target, ext_target, 20000)

## Feature Engineering

In [ ]:
# 1: Missing Core/Extended case ID
# Attempt 1: set to 0 if NaN

v_sample.CoreCaseGraphID = v_sample.CoreCaseGraphID.fillna(0)
v_sample.ExtendedCaseGraphID = v_sample.ExtendedCaseGraphID.fillna(0)

In [ ]:
# Check how many nodes I have

print(f"Nodes: {v_sample.count().values[0]} with {e_sample.count().values[0]} edges")

In [ ]:
# Create dataframes for each node type (Account, Customer, Derived entity, External entity, Address)
# and each edge type (has account, has address, is similar, money transfer)

v_sets = defaultdict()
for v_type in list(pd.Categorical(v_sample.Label).categories):
    v_sets[v_type] = v_sample[v_sample.Label == v_type]
    v_sets[v_type] = v_sets[v_type].drop(['Label', 'testingFlag']+list(v_sets[v_type].columns[v_sets[v_type].isnull().all()]), axis=1)

    e_sets = defaultdict()
for e_type in list(pd.Categorical(e_sample.Label).categories):
    e_sets[e_type] = e_sample[e_sample.Label == e_type]
    e_sets[e_type] = e_sets[e_type].drop(['Label']+list(e_sets[e_type].columns[e_sets[e_type].isnull().all()]), axis=1)
    e_sets[e_type] = e_sets[e_type].rename(columns={'from_id':'source', 'to_id':'target'})

In [ ]:
# 2: Logical conversion from string to numeric (for categorical features)

#Revenue Size Flag: low, mid_low, medium, mid_high, high -> 1,2,3,4,5
conversion = {'low':1, 'mid_low':2, 'medium':3, 'mid_high':4, 'high':5}
for i in v_sets:
  if 'Revenue Size Flag' in list(v_sets[i].columns):
    v_sets[i]['Revenue Size Flag']=v_sets[i]['Revenue Size Flag'].map(conversion)
    
#Income Size Flag: low, medium, high -> 1,2,3
conversion = {'low':1, 'medium':2, 'high':3}
for i in v_sets:
  if 'Income Size Flag' in list(v_sets[i].columns):
    v_sets[i]['Income Size Flag']=v_sets[i]['Income Size Flag'].map(conversion)

#Similarity Strength: weak, medium, strong -> 1,2,3
conversion = {'weak':1, 'medium':2, 'strong':3}
for i in e_sets:
  if 'Similarity Strength' in list(e_sets[i].columns):
    e_sets[i]['Similarity Strength']= e_sets[i]['Similarity Strength'].map(conversion)
    e_sets[i] = e_sets[i].rename(columns={'Similarity Strength':'weight'})

#Amount Flag: small, medium, large -> 1,50,500 -> treated as weights
conversion = {'small':1, 'medium':50, 'large':500}
for i in e_sets:
  if 'Amount Flag' in list(e_sets[i].columns):
    e_sets[i]['Amount Flag']=e_sets[i]['Amount Flag'].map(conversion)
    e_sets[i] = e_sets[i].rename(columns={'Amount Flag':'weight'})
    

In [ ]:
# 3: One-hot encoding for categorical features

for i in v_sets:
    if 'Person or Organisation' in list(v_sets[i].columns):
        v_sets[i] = pd.get_dummies(v_sets[i], columns=['Person or Organisation'])


In [ ]:
# 4: String features
# Attempt 1: remove them

for i in v_sets:
    if 'Account ID String' in list(v_sets[i].columns):
      v_sets[i] = v_sets[i].drop('Account ID String', axis=1)
    if 'Address' in list(v_sets[i].columns):
      v_sets[i] = v_sets[i].drop('Address', axis=1)
    if 'Name' in list(v_sets[i].columns):
      v_sets[i] = v_sets[i].drop('Name', axis=1)


## StellarGraph setup

In [ ]:
G = sg.StellarDiGraph(v_sets, e_sets)

In [ ]:
print(G.info())

In [ ]:
#### Graph embedding with NODE2VEC and WORD2VEC

rw = sg.data.BiasedRandomWalk(G)
t0 = time.time()
walks = rw.run(
    nodes=list(G.nodes()),  # root nodes
    length=10,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.6,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=1.7,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
t1 = time.time()
print("Number of random walks: {} in {:.2f} s".format(len(walks), (t1-t0)))


from gensim.models import Word2Vec
str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=128, window=5, min_count=0, sg=1, workers=8, iter=5)
# size: length of embedding vector

# The embedding vectors can be retrieved from model.wv using the node ID.
# model.wv["19231"].shape 
  
# Retrieve node embeddings 
node_ids = model.wv.index2word  # list of node IDs
node_embeddings = (model.wv.vectors)  # numpy.ndarray of size number of nodes times embeddings dimensionality

# Retrieve corresponding targets

# from training csv
# core_targets = core_target_sample.loc[[int(node_id) for node_id in node_ids if int(node_id) in list(core_target_sample.index)]].CaseID
# ext_targets = ext_target_sample.loc[[int(node_id) for node_id in node_ids if int(node_id) in list(ext_target_sample.index)]].CaseID

# from vertices' data
core_targets = v_sample.loc[[int(node_id) for node_id in node_ids]].CoreCaseGraphID
ext_targets = v_sample.loc[[int(node_id) for node_id in node_ids]].ExtendedCaseGraphID


In [ ]:
model.wv.vectors.shape

In [ ]:
ext_targets.shape

In [ ]:
# Take label of nodes by node_id

tlab = time.time()
labels = [None] * len(node_ids)
for i in range(len(node_ids)):
    labels[i] = v_sample.loc[[int(node_ids[i])]].Label.item()
tlab2 = time.time()
print(f"Done {len(node_ids)} passes in {(tlab2-tlab):.2f} s")

In [ ]:
# Transform the embeddings to 2d space for visualization
transform = TSNE #PCA
trans = transform(n_components=2)
t2 = time.time()
node_embeddings_2d_tsne = trans.fit_transform(node_embeddings)
t3 = time.time()
print("dimensionality reduction done in {:.2f} s".format(t3-t2))

In [ ]:
# Draw the embedding points, coloring them by the target label (CaseID)
alpha = 0.6
label_map = {l: i for i, l in enumerate(np.unique(ext_targets), start=1000) if pd.notna(l)}
label_map[0] = 1
node_colours = [label_map[target] if pd.notna(target) else 0 for target in ext_targets]

plt.figure(figsize=(15, 15))
plt.axes().set(aspect="equal")
plt.scatter(
    node_embeddings_2d_tsne[:, 0],
    node_embeddings_2d_tsne[:, 1],
    c=node_colours,
    cmap="jet",
    alpha=alpha,
)
plt.title("{} visualization of node embeddings w.r.t. Extended Case ID".format(transform.__name__))
plt.show()

In [ ]:
# Draw the embedding points, coloring them by node label
alpha = 0.6
label_map = {l: i*500 for i, l in enumerate(np.unique(labels), start=10) if pd.notna(l)}
node_colours = [label_map[label] for label in labels]
#node_colours = [label_map[target] if pd.notna(target) else 0 for target in ext_targets]

plt.figure(figsize=(15, 15))
plt.axes().set(aspect="equal")
plt.scatter(
    node_embeddings_2d_tsne[:, 0],
    node_embeddings_2d_tsne[:, 1],
    c=node_colours,
    cmap="BuGn",
    alpha=alpha,
)
plt.title("{} visualization of node embeddings w.r.t. Extended Case ID".format(transform.__name__))
plt.show()

In [ ]:
# Transform the embeddings to 2d space for visualization
transform = PCA
trans = transform(n_components=2)
node_embeddings_2d_pca = trans.fit_transform(node_embeddings)

# Draw the embedding points, coloring them by the target label (CaseID)
alpha = 0.6
label_map = {l: i for i, l in enumerate(np.unique(ext_targets), start=10) if pd.notna(l)}
node_colours = [label_map[target] if pd.notna(target) else 0 for target in ext_targets]

plt.figure(figsize=(15, 15))
plt.axes().set(aspect="equal")
plt.scatter(
    node_embeddings_2d_pca[:, 0],
    node_embeddings_2d_pca[:, 1],
    c=node_colours,
    cmap="ocean",
    alpha=alpha,
)
plt.title("{} visualization of node embeddings w.r.t. Extended Case ID".format(transform.__name__))
plt.show()

In [ ]:
# PCA here is useless
# This approach produces very low quality embeddings